# MultiAgentRL Colab Runner (복구본)
이 노트북은 Colab에서 `runMARL.py`, `rungnn.py`, `rungnnpool.py`를 안정적으로 실행하고 Drive 백업까지 수행하는 워크플로우입니다.

사용 순서:
1. 1~7번 셀(환경 준비) 실행
2. 8/9/10번 셀은 서로 독립 실행입니다(한 번에 하나만 `RUN_*=True` 권장)
3. 각 셀은 학습 중 `--backup-every` 주기마다 Drive에 중간 체크포인트를 저장합니다
4. 11번 셀로 결과 확인


In [ ]:
# 이 셀은 레포를 Colab에 동기화하고 최신 코드로 맞춥니다.
REPO_URL = "https://github.com/jisang0706/MultiAgentRL_InventoryControl.git"
REPO_DIR = "/content/MultiAgentRL_InventoryControl"

import os

if os.path.exists(REPO_DIR):
    %cd {REPO_DIR}
    !git fetch origin
    !git checkout master
    !git reset --hard origin/master
else:
    !git clone {REPO_URL} {REPO_DIR}
    %cd {REPO_DIR}

!pwd
!git rev-parse --short HEAD
!ls -1 | head


In [ ]:
# 이 셀은 Colab 충돌을 줄이는 핵심 패키지를 설치합니다.
!pip -q install -U pip setuptools wheel
!pip -q install "jedi>=0.19.1"
!pip -q install "ray[rllib]==2.31.0" "omegaconf==2.3.0" "json5==0.9.25


In [ ]:
# 이 셀은 현재 torch 버전에 맞춰 torch-geometric 관련 패키지를 설치합니다.
import subprocess, sys
import torch

torch_v = torch.__version__.split("+")[0]
cuda = torch.version.cuda
tag = f"cu{cuda.replace('.', '')}" if cuda else "cpu"
wheel = f"https://data.pyg.org/whl/torch-{torch_v}+{tag}.html"

print("torch:", torch.__version__)
print("pyg wheel:", wheel)

pkgs = ["pyg_lib", "torch_scatter", "torch_sparse", "torch_cluster", "torch_spline_conv"]
cmd = [sys.executable, "-m", "pip", "install", "-q", *pkgs, "-f", wheel]
rc = subprocess.run(cmd).returncode
if rc != 0:
    print("compiled wheel install failed; continue with base torch-geometric install")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "torch-geometric==2.5.3"])
print("torch-geometric install done")


In [ ]:
# 이 셀은 Google Drive를 연결하고 학습 산출물을 백업하는 함수를 정의합니다.
from google.colab import drive
from pathlib import Path
import shutil
import time
import re

# 이미 마운트되어 있어도 재호출해도 동작함
drive.mount('/content/drive')

BACKUP_BASE = Path('/content/drive/MyDrive/MultiAgentRL_InventoryControl_backup')
BACKUP_BASE.mkdir(parents=True, exist_ok=True)
print('BACKUP_BASE =', BACKUP_BASE)

def _copytree_if_exists(src: Path, dst: Path):
    if src.exists():
        shutil.copytree(src, dst, dirs_exist_ok=True)
        return True
    return False

def backup_artifacts(run_tag: str, log_path: str | None = None):
    ts = time.strftime('%Y%m%d_%H%M%S')
    run_dir = BACKUP_BASE / f"{run_tag}_{ts}"
    ckpt_dir = run_dir / 'checkpoints'
    logs_dir = run_dir / 'logs'
    meta_dir = run_dir / 'meta'
    run_dir.mkdir(parents=True, exist_ok=True)
    ckpt_dir.mkdir(parents=True, exist_ok=True)
    logs_dir.mkdir(parents=True, exist_ok=True)
    meta_dir.mkdir(parents=True, exist_ok=True)

    # 1) ray_results 백업
    copied = 0
    copied += _copytree_if_exists(Path('/root/ray_results'), run_dir / 'ray_results_root')
    copied += _copytree_if_exists(Path('/content/MultiAgentRL_InventoryControl/ray_results'), run_dir / 'ray_results_repo')

    # 2) 로그 파일 백업
    if log_path and Path(log_path).exists():
        shutil.copy2(log_path, logs_dir / Path(log_path).name)

        # 3) 로그에서 /tmp/tmp* 체크포인트 경로 추출 후 백업
        text = Path(log_path).read_text(errors='ignore')
        paths = sorted(set(re.findall(r"/tmp/tmp[\w\-]+", text)))
        for p in paths:
            src = Path(p)
            if src.exists():
                _copytree_if_exists(src, ckpt_dir / src.name)

    # 4) 실행 재현용 코드 스냅샷 백업
    for fn in ['runMARL.py', 'rungnn.py', 'rungnnpool.py', 'checkpoint_backup.py', 'env3rundiv.py', 'ccmodel.py', 'model.py', 'modelpool.py', 'main.ipynb']:
        f = Path('/content/MultiAgentRL_InventoryControl') / fn
        if f.exists():
            shutil.copy2(f, meta_dir / fn)

    print('backup done ->', run_dir)
    return str(run_dir)


In [ ]:
# 이 셀은 설치된 핵심 버전이 정상인지 확인합니다.
import sys, ray, gymnasium, numpy, torch
print('python:', sys.version)
print('ray:', ray.__version__)
print('gymnasium:', gymnasium.__version__)
print('numpy:', numpy.__version__)
print('torch:', torch.__version__)


In [ ]:
# 이 셀은 Ray/스크립트 호환 패치를 자동 적용합니다(안전하게 여러 번 실행 가능).
from pathlib import Path

# runMARL qmix import 제거 + env_config 보강(과거 코드 대비)
f = Path('runMARL.py')
if f.exists():
    txt = f.read_text()
    needle_import = 'from ray.rllib.algorithms.qmix import QMixConfig\n'
    if needle_import in txt:
        txt = txt.replace(needle_import, '')
        print('runMARL: removed qmix import')

    old_env = 'env_config={\n            "num_agents": num_agents,\n        },'
    new_env = 'env_config={\n            "connections": config["connections"],\n            "num_nodes": num_nodes,\n            "num_agents": num_agents,\n        },'
    if old_env in txt:
        txt = txt.replace(old_env, new_env)
        print('runMARL: patched env_config')
    f.write_text(txt)

# rungnn/rungnnpool env_config 보강(과거 코드 대비)
for script in ['rungnn.py', 'rungnnpool.py']:
    f = Path(script)
    if not f.exists():
        continue
    txt = f.read_text()
    old_env = 'env_config={\n            "num_agents": num_agents,\n        },'
    new_env = 'env_config={\n            "connections": config["connections"],\n            "num_nodes": num_nodes,\n            "num_agents": num_agents,\n        },'
    if old_env in txt:
        txt = txt.replace(old_env, new_env)
        print(f'{script}: patched env_config')
    f.write_text(txt)

print('compat patch done')


In [ ]:
# 이 셀은 환경이 정상 임포트/초기화되는지 빠르게 점검합니다.
from env3rundiv import MultiAgentInvManagementDiv

cfg = {
    "connections": {0: [1, 2], 1: [3, 4], 2: [4, 5], 3: [], 4: [], 5: []},
    "num_nodes": 6,
}

env = MultiAgentInvManagementDiv(cfg)
obs, infos = env.reset()
print('num agents:', len(obs))
print('agent ids sample:', list(obs.keys())[:3])
print('obs shape:', env.observation_space.shape, 'act shape:', env.action_space.shape)


In [ ]:
# 이 셀은 중앙 critic(runMARL)을 단독 학습하고 중간 체크포인트를 주기적으로 Drive에 백업합니다.
RUN_MAIN = False   # 이 셀만 실행하려면 True
ITER_MAIN = 60     # 빠른 검증은 3~5 권장
BACKUP_EVERY_MAIN = 5
LOG_MAIN = '/content/runMARL.log'
CKPT_DIR_MAIN = '/content/drive/MyDrive/MultiAgentRL_InventoryControl_backup/live_ckpt/runMARL'
RESTORE_MAIN = ''  # 이어서 학습하려면 이전 체크포인트 경로 입력

if RUN_MAIN:
    cmd = f"python runMARL.py --iterations {ITER_MAIN} --backup-dir {CKPT_DIR_MAIN} --backup-every {BACKUP_EVERY_MAIN}"
    if RESTORE_MAIN.strip():
        cmd += f" --restore-checkpoint '{RESTORE_MAIN}'"
    !{cmd} 2>&1 | tee {LOG_MAIN}
    backup_artifacts(f'runMARL_iter{ITER_MAIN}', LOG_MAIN)
else:
    print('RUN_MAIN=False 이므로 스킵했습니다. 실행하려면 True로 바꾸세요.')


In [ ]:
# 이 셀은 GNN(rungnn)을 단독 학습하고 중간 체크포인트를 주기적으로 Drive에 백업합니다.
RUN_GNN = False    # 이 셀만 실행하려면 True
ITER_GNN = 60      # 빠른 검증은 3~5 권장
BACKUP_EVERY_GNN = 5
LOG_GNN = '/content/rungnn.log'
CKPT_DIR_GNN = '/content/drive/MyDrive/MultiAgentRL_InventoryControl_backup/live_ckpt/rungnn'
RESTORE_GNN = ''   # 이어서 학습하려면 이전 체크포인트 경로 입력

if RUN_GNN:
    cmd = f"python rungnn.py --iterations {ITER_GNN} --backup-dir {CKPT_DIR_GNN} --backup-every {BACKUP_EVERY_GNN}"
    if RESTORE_GNN.strip():
        cmd += f" --restore-checkpoint '{RESTORE_GNN}'"
    !{cmd} 2>&1 | tee {LOG_GNN}
    backup_artifacts(f'rungnn_iter{ITER_GNN}', LOG_GNN)
else:
    print('RUN_GNN=False 이므로 스킵했습니다. 실행하려면 True로 바꾸세요.')


In [ ]:
# 이 셀은 GNNPool(rungnnpool)을 단독 학습하고 중간 체크포인트를 주기적으로 Drive에 백업합니다.
RUN_GNNPOOL = False   # 이 셀만 실행하려면 True
ITER_GNNPOOL = 60     # 빠른 검증은 3~5 권장
BACKUP_EVERY_GNNPOOL = 5
LOG_GNNPOOL = '/content/rungnnpool.log'
CKPT_DIR_GNNPOOL = '/content/drive/MyDrive/MultiAgentRL_InventoryControl_backup/live_ckpt/rungnnpool'
RESTORE_GNNPOOL = ''  # 이어서 학습하려면 이전 체크포인트 경로 입력

if RUN_GNNPOOL:
    cmd = f"python rungnnpool.py --iterations {ITER_GNNPOOL} --backup-dir {CKPT_DIR_GNNPOOL} --backup-every {BACKUP_EVERY_GNNPOOL}"
    if RESTORE_GNNPOOL.strip():
        cmd += f" --restore-checkpoint '{RESTORE_GNNPOOL}'"
    !{cmd} 2>&1 | tee {LOG_GNNPOOL}
    backup_artifacts(f'rungnnpool_iter{ITER_GNNPOOL}', LOG_GNNPOOL)
else:
    print('RUN_GNNPOOL=False 이므로 스킵했습니다. 실행하려면 True로 바꾸세요.')


In [ ]:
# 이 셀은 백업된 result.json에서 학습 곡선을 자동으로 찾아서 보여줍니다.
import os, glob, json
import pandas as pd
import matplotlib.pyplot as plt

cands = (
    glob.glob('/root/ray_results/**/result.json', recursive=True) +
    glob.glob('/content/MultiAgentRL_InventoryControl/ray_results/**/result.json', recursive=True) +
    glob.glob('/content/drive/MyDrive/**/ray_results/**/result.json', recursive=True)
)
cands = [p for p in cands if os.path.exists(p) and os.path.getsize(p) > 0]
cands = sorted(cands, key=os.path.getmtime, reverse=True)

if not cands:
    raise FileNotFoundError('사용 가능한 result.json이 없습니다.')

def load_df(path):
    records = []
    with open(path, 'r') as f:
        for ln in f:
            ln = ln.strip()
            if not ln:
                continue
            try:
                row = json.loads(ln)
            except Exception:
                continue
            if not isinstance(row, dict):
                continue

            m = row.get('metrics', row)
            if not isinstance(m, dict):
                continue
            er = m.get('env_runners', {}) if isinstance(m.get('env_runners', {}), dict) else {}

            ti = m.get('training_iteration', m.get('iteration', m.get('iterations_since_restore')))
            tt = m.get('time_total_s', m.get('time_since_restore'))
            ep_mean = m.get('episode_reward_mean', er.get('episode_reward_mean'))
            ep_min = m.get('episode_reward_min', er.get('episode_reward_min'))
            ep_max = m.get('episode_reward_max', er.get('episode_reward_max'))

            records.append({
                'training_iteration': ti,
                'time_total_s': tt,
                'ep_mean': ep_mean,
                'ep_min': ep_min,
                'ep_max': ep_max,
            })

    df = pd.DataFrame(records, columns=['training_iteration','time_total_s','ep_mean','ep_min','ep_max'])
    if df.empty:
        return df
    df['training_iteration'] = pd.to_numeric(df['training_iteration'], errors='coerce')
    df = df.dropna(subset=['training_iteration']).sort_values('training_iteration')
    return df

used = None
df = pd.DataFrame()
for p in cands:
    tmp = load_df(p)
    if not tmp.empty:
        used = p
        df = tmp
        break

if used is None:
    raise RuntimeError('파싱 가능한 학습 로그가 없습니다.')

print('using:', used)
display(df[['training_iteration','ep_mean','ep_min','ep_max','time_total_s']].tail())

plt.figure(figsize=(7,4))
plt.plot(df['training_iteration'], df['ep_mean'])
plt.xlabel('iteration')
plt.ylabel('episode_reward_mean')
plt.grid(True)
plt.show()
